<a href="https://colab.research.google.com/github/wenchun0731/crosswalk_identify/blob/main/crossWalkConnect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
from google.colab.patches import cv2_imshow
from sklearn.cluster import DBSCAN
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [ ]:
class find_up_and_low_slope:
    def __init__(self):
      pass

    def main_findupandlow_slope(self,numpy_point):
      pointx,pointy=self.change_to_simple_data(numpy_point)
      return self.coefficients_line(pointx,pointy)

    #輸出上下線的pointx,pointy
    def change_to_simple_data(self,numpy_point):
      simple_data=[[point.tolist() for point in points] for points in numpy_point]
      pointx=[point[0][0] for points in simple_data for point in points]
      pointy=[point[0][1] for points in simple_data for point in points]
      return pointx,pointy

    def coefficients_line(self,pointx,pointy):
      coefficients = np.polyfit(pointx,pointy, 1)
      linear_fit = np.poly1d(coefficients)
      slope=round(coefficients[0],3)
      intercept=round(coefficients[1],3)
      return slope,intercept

class identify_crosswalk:
  def __init__(self,image):
    self.image=image




  def main_crosswalk(self):
    gray_image=self.opencv_gray()
    binary_image=self.opencv_binary(gray_image)
    point_all,approx_all=self.opencv_contours(binary_image)
    contours_sort,slope=self.opencv_dbscan(point_all,approx_all)
    up_line_information,low_line_information=self.slope_judge(contours_sort,slope[0])
    return point_all,up_line_information,low_line_information

  def opencv_gray(self):
    gray_img = cv2.cvtColor(self.image,cv2.COLOR_BGR2GRAY)
    return gray_img

  def opencv_binary(self,image):
    threshold_value = 180  # 可以根據需要調整
    max_value = 255
    _, binary_image = cv2.threshold(image, threshold_value, max_value, cv2.THRESH_BINARY)
    return binary_image

  def opencv_contours(self,image):
    contours, hierarchy = cv2.findContours(image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    # 所有要得輪廓
    approx_all=[]
    # 所有要得中心點
    point_all=[]

    for contour in contours:
        #計算面積，看面積有沒有大於1500

        area=cv2.contourArea(contour)
        if area>1500:

          # 使用多邊形逼近法來取得簡化的頂點
          epsilon = 0.01 * cv2.arcLength(contour, True)
          approx = cv2.approxPolyDP(contour, epsilon, True)

          # 檢查是否為四邊形
          if len(approx) == 4:
              # 計算邊的長度比和角度，確認是否為平行四邊形
              # 這裡可以加入計算平行邊的判斷邏輯
              M = cv2.moments(contour)
              if M['m00'] != 0:
                  cx = int(M['m10'] / M['m00'])
                  cy = int(M['m01'] / M['m00'])

                  point_all.append([cx,cy])
                  approx_all.append(approx)
    return point_all,approx_all

  #找到要得中心點的擬合直線，使用斜率判斷contour_line_sort演算法
  def opencv_dbscan(self,point_all,approx_all):
    pointx=[]
    pointy=[]
    contours_sort=[] # 要保留的輪廓
    # 設定 DBSCAN 參數
    epsilon = 200  # 鄰域距離閾值，視情況調整
    min_points = 3  # 至少包含的點數

    # 初始化 DBSCAN 模型
    dbscan = DBSCAN(eps=epsilon, min_samples=min_points)
    clusters = dbscan.fit_predict(point_all)

    # 輸出結果
    for i, cluster_id in enumerate(clusters):
        x,y=point_all[i]
        if cluster_id != -1:  # -1 表示噪聲
            # cv2.circle(image, (x, y), 10, (255, 0, 0), -1)
            pointx.append(x)
            pointy.append(y)

            contours_sort.append(approx_all[i])
    slope=find_up_and_low_slope()
    slope=slope.coefficients_line(pointx,pointy)
    return contours_sort,slope

  #找到上下線的點點，輸出斜率與截距
  def contour_line_sort(self,contour_sort,collabor):
    low_line_point=[]
    up_line_point=[]
    sorted_array = [sorted(myarray, key=lambda x: x[0][collabor]) for myarray in contour_sort]
    for contour in sorted_array:
      low_line_point.append(contour[:2])
      up_line_point.append(contour[2:])

    up=find_up_and_low_slope()
    up_line_slope=up.main_findupandlow_slope(up_line_point)
    low=find_up_and_low_slope()
    low_line_slope=low.main_findupandlow_slope(low_line_point)

    return up_line_slope,low_line_slope



  # 以y軸區分演算法



  def slope_judge(self,contour_sort,slope):
    if abs(slope)>=0.3:
      #以x軸區分的演算法
      return self.contour_line_sort(contour_sort,0)
    else:
      #以y軸區分的演算法
      return self.contour_line_sort(contour_sort,1)